In [98]:
import gzip
import pandas as pd
import numpy as np
import datetime

In [99]:
df = pd.read_csv('data_cleaned.csv')

In [100]:
df.shape

(19393788, 9)

In [101]:
df.head()

,timestamp,node_id,subsystem,sensor,parameter,value_hrf,hrf_unit,hrf_minval,hrf_maxval
0,2019/02/04 00:00:01,001e0610e532,lightsense,apds_9006_020,intensity,0.241,lux,0.0,1000.0
1,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,humidity,41.090,RH,0.0,100.0
2,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,temperature,26.190,C,-25.0,85.0
3,2019/02/04 00:00:01,001e0610e532,lightsense,mlx75305,intensity,2.032,uW/cm^2,0.0,160.0
4,2019/02/04 00:00:01,001e0610e532,lightsense,tmp421,temperature,23.590,C,-55.0,127.0


In [105]:
def conc_fixer(row):
    if row['parameter'] == 'concentration':
        return row['sensor']
    else:
        return row['parameter']

In [106]:
df['parameter'] = df.apply(lambda x : conc_fixer(x), axis = 1)
    

In [107]:
df.drop(columns = ['hrf_minval', 'hrf_maxval'], inplace = True)

In [108]:
df = df.groupby(by = ['timestamp', 'node_id','parameter','hrf_unit']).mean().reset_index()

In [109]:
df.head()

,timestamp,node_id,parameter,hrf_unit,value_hrf
0,2019/02/04 00:00:01,001e0610e532,humidity,RH,60.093333
1,2019/02/04 00:00:01,001e0610e532,intensity,lux,0.241000
2,2019/02/04 00:00:01,001e0610e532,intensity,uW/cm^2,0.570000
3,2019/02/04 00:00:01,001e0610e532,pressure,hPa,992.930000
4,2019/02/04 00:00:01,001e0610e532,temperature,C,16.155714


In [110]:
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))

In [111]:
df.set_index('timestamp', inplace = True)

In [112]:
df.head()

,node_id,parameter,hrf_unit,value_hrf
timestamp,,,,
2019-02-04 00:00:01,001e0610e532,humidity,RH,60.093333
2019-02-04 00:00:01,001e0610e532,intensity,lux,0.241000
2019-02-04 00:00:01,001e0610e532,intensity,uW/cm^2,0.570000
2019-02-04 00:00:01,001e0610e532,pressure,hPa,992.930000
2019-02-04 00:00:01,001e0610e532,temperature,C,16.155714


In [113]:
df1 = df[df['hrf_unit'] != 'count']
df2 = df[df['hrf_unit'] == 'count']

In [114]:
df1 = df1.groupby([df1.index.day, df1.index.hour, 'node_id', 'parameter', 'hrf_unit']).mean()
df2 = df2.groupby([df2.index.day, df2.index.hour, 'node_id', 'parameter', 'hrf_unit']).sum()

In [115]:
df1.reset_index(level=[2,3,4], inplace = True)
df2.reset_index(level=[2,3,4], inplace = True)

In [116]:
df1.reset_index(level= 1, inplace = True)
df2.reset_index(level= 1, inplace = True)

In [117]:
 df = pd.concat([df1, df2])

In [118]:
df.index = df.index.map(lambda x: '2019-02-'+str(x) if len(str(x)) > 1 else '2019-02-0'+str(x))

In [119]:
df.columns = ['hour', 'node_id', 'parameter', 'hrf_unit', 'value_hrf']

In [126]:
df.tail()

,date,hour,node_id,parameter,hrf_unit,value_hrf
79891,2019-02-10,21,001e0610ba46,person_total,count,35.0
79892,2019-02-10,22,001e0610ba46,car_total,count,0.0
79893,2019-02-10,22,001e0610ba46,person_total,count,13.0
79894,2019-02-10,23,001e0610ba46,car_total,count,0.0
79895,2019-02-10,23,001e0610ba46,person_total,count,98.0


In [121]:
df.reset_index(level= 0, inplace = True)

In [122]:
df.columns = ['date', 'hour', 'node_id', 'parameter', 'hrf_unit', 'value_hrf']

In [123]:
df.to_csv('data_cleaned_aggregated.csv')

In [127]:
df.shape

(79896, 6)